<a href="https://colab.research.google.com/github/nangchung/truyvandaphuong/blob/main/Vector_Space_inver_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
import numpy as np
import re


def load_data_in_a_directory(data_path):
    file_paths = glob.glob(data_path)
    lst_contents = []
    for file_path in file_paths:
        f = open(file_path, 'r',encoding="utf-8")
        str = f.read()
        words = str.replace('"', '').replace('.', '').replace("'","").split()
        #words = set(words)
        lst_contents.append(words)
    return (lst_contents, file_paths)

# Doc noi dung cua tung file txt
# va xay dung tap "dictionary" chua danh sach cac tu
def build_dictionary(contents):
    dictionary = set()
    for content in contents:
        dictionary.update(content)
    return dictionary


def calc_tf_weighting_inverted(vocab, contents):
    TFinverted = []
    
    for indexW, word in enumerate(vocab):
        files = []
        
        for indexT, title in enumerate(contents):
            file = []
            if word in title:
                file.append(indexT)
                file.append(title.count(word) / len(title))
                files.append(file)
#                 print(file)
        TFinverted.append(files) 

    return TFinverted

def calc_tf_weighting_inverted_query(vocab, contents, querys):
    TFinverted = []
    set_querys = set(querys)
    
        
    for query in set_querys:
        if (query not in vocab):
            continue;
        else:
            for indexW, word in enumerate(vocab):
                file = []
                if (query == word):
                    file.append(indexW)
                    file.append(querys.count(query) / len(querys))

            #                 print(file)
                    TFinverted.append(file) 

    return TFinverted


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# MAIN

# BUOC 1: Load cac file trong 'data' va xay dụng tap cac tu vung
contents, paths = load_data_in_a_directory('/content/drive/My Drive/truy van da phuong/filedata/*.txt')
vocab = build_dictionary(contents)


In [8]:
# BUOC 2: Xay dung vector TF weighting cho 
# tap van ban va truy van
TF = calc_tf_weighting_inverted(vocab, contents)
query="TP"
qcontent = query.split()
qTF = calc_tf_weighting_inverted_query(vocab, contents, qcontent)


In [9]:
# BUOC 3: Xay dung vector IDF weight cho tap van ban
DF = np.array([len(x) for x in TF])
IDF = 1 + np.log(len(contents) / DF)
IDF = np.array([IDF]).T
IDF

array([[7.90975328],
       [4.81871083],
       [1.66171041],
       ...,
       [7.90975328],
       [6.52345892],
       [7.90975328]])

In [10]:
# BUOC 4: Xay dung vector TF_IDF weighting cho
# tap van ban va truy van
for i in range(len(IDF)):
    for j in range(len(TF[i])):
#         print(TF2[i])
#         print("\n")
        TF[i][j][1] = TF[i][j][1] * IDF[i]
for q in qTF:
    q[1] *= IDF[q[0]]


In [11]:
# BUOC 5: Tinh do tuong dong cua query va cac van ban
# su dung TF_IDF weighting
dists = np.zeros(len(contents))

for q in qTF:
    for tf_idf in TF[q[0]]:
        dists[tf_idf[0]] += (q[1] - tf_idf[1]) ** 2
        
# BUOC 6: Sap xep de sap hang va hien thi ket qua
rank = np.argsort(dists)[::-1]
print(rank)
print(dists)
topK = 5
for i in range(topK):
    print('Van ban gan thu ', i+1, ' la: ', ' '.join(paths[rank[i]]))

[551 385 106 ... 655 654   0]
[0. 0. 0. ... 0. 0. 0.]
Van ban gan thu  1  la:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 9 9 0 . t x t
Van ban gan thu  2  la:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 8 2 4 . t x t
Van ban gan thu  3  la:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 5 4 5 . t x t
Van ban gan thu  4  la:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 8 5 5 . t x t
Van ban gan thu  5  la:  / c o n t e n t / d r i v e / M y   D r i v e / t r u y   v a n   d a   p h u o n g / f i l e d a t a / d a t a 4 0 5 . t x t
